In [16]:
#importing the libraries that are required 
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [17]:
#defines a function named markAttendance that appears to be used for marking attendance.
def markAttendance(name):
    attendance_file_path = 'D:\grow\multiface\Attendance.csv'  
    with open(attendance_file_path, 'r+') as f:
        myDataList = f.readlines()

In [18]:
#line sets the path variable to the directory path
path = 'D:\grow\multiface\Training_images'
#initializes an empty list called images. This list will be used to store image data
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['Jishnu.jpg']


In [19]:
#to load and process the images found in the Training_images
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Jishnu']


In [20]:
#code defines a function named findEncodings that appears to be designed for encoding face images using the face_recognition library.
def findEncodings(images):
    encodeList = []

    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [21]:
#code defines a function named markAttendance and then calls a function findEncodings
def markAttendance(name):
    with open('Attendance.csv', 'r+') as f:
        myDataList = f.readlines()

        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'{name},{dtString}\n')


encodeListKnown = findEncodings(images)
print('Encoding Complete')

Encoding Complete


In [22]:
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()

    # Checking wether the frame was captured successfully
    if not success:
        continue  

    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)

    cv2.imshow('Webcam', img)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

In [8]:
#This code continuously captures video frames from the webcam, detects and recognizes faces in each frame, and displays
# the frames with recognized names and bounding boxes around the faces. It also marks attendance for recognized 
#individuals using the markAttendance function.
# The loop continues until the 'q' key is pressed.